In [35]:
import os
import re
from skimage.filters import threshold_otsu,gaussian
from skimage import measure
import pandas as pd
import numpy as nm
from sklearn.preprocessing import MinMaxScaler
from skimage.io import imread
from skimage import color
from skimage.transform import resize
from numpy import asarray
import matplotlib.pyplot as plt

scaler = MinMaxScaler()

def image_to_dataframe(image):
    Lead_1 = image[300:600, 150:643]
    Lead_2 = image[300:600, 646:1135]
    Lead_3 = image[300:600, 1140:1626]
    Lead_4 = image[300:600, 1630:2125]
    Lead_5 = image[600:900, 150:643]
    Lead_6 = image[600:900, 646:1135]
    Lead_7 = image[600:900, 1140:1626]
    Lead_8 = image[600:900, 1630:2125]
    Lead_9 = image[900:1200, 150:643]
    Lead_10 = image[900:1200, 646:1135]
    Lead_11 = image[900:1200, 1140:1626]
    Lead_12 = image[900:1200, 1630:2125]
    Lead_13 = image[1250:1480, 150:2125]
    
    Leads=[Lead_1,Lead_2,Lead_3,Lead_4,Lead_5,Lead_6,Lead_7,Lead_8,Lead_9,Lead_10,Lead_11,Lead_12,Lead_13]
    binary_global_images = []
    df_final = pd.DataFrame()
    for x, y in enumerate(Leads):
        grayscale = color.rgb2gray(y)
        blurred_image = gaussian(grayscale, sigma=0.7)
        global_thresh = threshold_otsu(blurred_image)
        binary_global = blurred_image < global_thresh
        if x!=12:
            binary_global = resize(binary_global, (300, 450))
        binary_global_images.append(binary_global)
        contours = measure.find_contours(binary_global,0.8)
        contours_shape = sorted([x.shape for x in contours])[::-1][0:1]
        for contour in contours:
            if contour.shape in contours_shape:
                test = resize(contour, (255, 2))
        df_test = pd.DataFrame(test, columns = ['X','Y'])
        scaler = MinMaxScaler()
        fit_transform_data = scaler.fit_transform(test)
        Normalized_Scaled=pd.DataFrame(fit_transform_data[:,0], columns = ['X'])
        Normalized_Scaled=Normalized_Scaled.T
        df_final=pd.concat([df_final,Normalized_Scaled],axis=1,ignore_index=True)
    
    return df_final

(1, 3315)

In [41]:
df

,0,1,2,3,4,5,6,7,8,9,...,3305,3306,3307,3308,3309,3310,3311,3312,3313,3314
X,0.980192,1.0,0.938302,0.847854,0.753727,0.652048,0.553429,0.446343,0.349131,0.335356,...,0.891744,0.735017,0.707441,0.733016,0.647552,0.384545,0.136368,0.271427,0.582282,0.946965


In [51]:
def process_images_in_directory(directory, target):
    df_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            image_path = os.path.join(directory, filename)
            image = imread(image_path)
            df = image_to_dataframe(image)
            
            # Add a "target" column with the specified value
            df['target'] = target
            
            df_list.append(df)
    return pd.concat(df_list, axis=0)

# Directories
MI_dir = "/Users/omgoswami/Desktop/ECG_stuffs/train/ECG Images of Myocardial Infarction Patients (240x12=2880)"
Norm_dir = "/Users/omgoswami/Desktop/ECG_stuffs/train/Normal Person ECG Images (284x12=3408)"
Abnorm_dir = "/Users/omgoswami/Desktop/ECG_stuffs/train/ECG Images of Patient that have abnormal heartbeat (233x12=2796)"
History_MI_dir = "/Users/omgoswami/Desktop/ECG_stuffs/train/ECG Images of Patient that have History of MI (172x12=2064)"

final_df = pd.DataFrame()
directories = [(MI_dir, "MI"), (Norm_dir, "Normal"), (Abnorm_dir, "Abnormal"), (History_MI_dir, "History_MI")]

for directory, target in directories:
    df_result = process_images_in_directory(directory, target)
    final_df = pd.concat([final_df, df_result], axis=0)

# Display the final_df
final_df

,0,1,2,3,4,5,6,7,8,9,...,3306,3307,3308,3309,3310,3311,3312,3313,3314,target
X,0.629864,0.695484,0.768929,0.827265,0.884672,0.912639,0.905737,0.918265,0.926312,0.902156,...,0.751686,0.755811,0.760430,0.527864,0.349887,0.630855,0.872002,0.735464,0.885782,MI
X,0.781526,0.776657,0.739374,0.681770,0.678423,0.732785,0.744234,0.732009,0.668425,0.578894,...,0.770477,0.745241,0.560934,0.567957,0.509071,0.386246,0.412518,0.549272,0.911574,MI
X,0.872471,0.810556,0.733582,0.650916,0.574571,0.499497,0.428428,0.401816,0.428237,0.470861,...,0.907343,0.896816,0.721197,0.442446,0.645074,0.920683,0.965847,0.706012,0.790617,MI
X,1.000000,0.983856,0.980918,0.980689,0.962271,0.886983,0.786168,0.678476,0.578183,0.616658,...,0.966950,0.972317,0.926683,0.705599,0.719133,0.901232,0.878187,0.648030,0.862759,MI
X,0.643017,0.691223,0.728113,0.752938,0.729519,0.659402,0.574370,0.474245,0.402940,0.461294,...,0.706268,0.650218,0.727416,0.746537,0.766283,0.514789,0.453669,0.783242,1.000000,MI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X,0.837912,0.841972,0.834226,0.824933,0.831612,0.839788,0.870319,0.903772,0.898621,0.864341,...,0.865397,0.904360,0.899706,0.823247,0.759462,0.871792,0.843129,0.648951,0.868968,History_MI
X,0.899518,0.880000,0.910408,0.940430,0.971388,0.982522,1.000000,0.991855,0.951647,0.871412,...,0.716792,0.626567,0.738240,0.610874,0.727605,0.757862,0.657690,0.792556,1.000000,History_MI
X,0.710817,0.720143,0.704892,0.617803,0.528379,0.414255,0.327710,0.363878,0.465357,0.553734,...,0.665386,0.824555,0.954049,0.786360,0.568688,0.665561,0.715077,0.735498,0.935721,History_MI
X,0.853779,0.741087,0.661940,0.728122,0.807062,0.846544,0.849598,0.851051,0.886479,0.899977,...,0.564531,0.756740,0.831966,0.748112,0.770145,0.705628,0.857763,0.867560,0.873760,History_MI


In [52]:
df_copy = final_df.copy()

In [53]:
df_copy

,0,1,2,3,4,5,6,7,8,9,...,3306,3307,3308,3309,3310,3311,3312,3313,3314,target
X,0.629864,0.695484,0.768929,0.827265,0.884672,0.912639,0.905737,0.918265,0.926312,0.902156,...,0.751686,0.755811,0.760430,0.527864,0.349887,0.630855,0.872002,0.735464,0.885782,MI
X,0.781526,0.776657,0.739374,0.681770,0.678423,0.732785,0.744234,0.732009,0.668425,0.578894,...,0.770477,0.745241,0.560934,0.567957,0.509071,0.386246,0.412518,0.549272,0.911574,MI
X,0.872471,0.810556,0.733582,0.650916,0.574571,0.499497,0.428428,0.401816,0.428237,0.470861,...,0.907343,0.896816,0.721197,0.442446,0.645074,0.920683,0.965847,0.706012,0.790617,MI
X,1.000000,0.983856,0.980918,0.980689,0.962271,0.886983,0.786168,0.678476,0.578183,0.616658,...,0.966950,0.972317,0.926683,0.705599,0.719133,0.901232,0.878187,0.648030,0.862759,MI
X,0.643017,0.691223,0.728113,0.752938,0.729519,0.659402,0.574370,0.474245,0.402940,0.461294,...,0.706268,0.650218,0.727416,0.746537,0.766283,0.514789,0.453669,0.783242,1.000000,MI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X,0.837912,0.841972,0.834226,0.824933,0.831612,0.839788,0.870319,0.903772,0.898621,0.864341,...,0.865397,0.904360,0.899706,0.823247,0.759462,0.871792,0.843129,0.648951,0.868968,History_MI
X,0.899518,0.880000,0.910408,0.940430,0.971388,0.982522,1.000000,0.991855,0.951647,0.871412,...,0.716792,0.626567,0.738240,0.610874,0.727605,0.757862,0.657690,0.792556,1.000000,History_MI
X,0.710817,0.720143,0.704892,0.617803,0.528379,0.414255,0.327710,0.363878,0.465357,0.553734,...,0.665386,0.824555,0.954049,0.786360,0.568688,0.665561,0.715077,0.735498,0.935721,History_MI
X,0.853779,0.741087,0.661940,0.728122,0.807062,0.846544,0.849598,0.851051,0.886479,0.899977,...,0.564531,0.756740,0.831966,0.748112,0.770145,0.705628,0.857763,0.867560,0.873760,History_MI


In [58]:
final_df.to_csv('final_df.csv',index=False)

In [59]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_column = label_encoder.fit_transform(df_copy['target'])
label_mapping_festival = {original_value: encoded_value for original_value, encoded_value in zip(df_copy['target'], encoded_column)}
df_copy['target'] = encoded_column
df_copy

,0,1,2,3,4,5,6,7,8,9,...,3306,3307,3308,3309,3310,3311,3312,3313,3314,target
X,0.629864,0.695484,0.768929,0.827265,0.884672,0.912639,0.905737,0.918265,0.926312,0.902156,...,0.751686,0.755811,0.760430,0.527864,0.349887,0.630855,0.872002,0.735464,0.885782,2
X,0.781526,0.776657,0.739374,0.681770,0.678423,0.732785,0.744234,0.732009,0.668425,0.578894,...,0.770477,0.745241,0.560934,0.567957,0.509071,0.386246,0.412518,0.549272,0.911574,2
X,0.872471,0.810556,0.733582,0.650916,0.574571,0.499497,0.428428,0.401816,0.428237,0.470861,...,0.907343,0.896816,0.721197,0.442446,0.645074,0.920683,0.965847,0.706012,0.790617,2
X,1.000000,0.983856,0.980918,0.980689,0.962271,0.886983,0.786168,0.678476,0.578183,0.616658,...,0.966950,0.972317,0.926683,0.705599,0.719133,0.901232,0.878187,0.648030,0.862759,2
X,0.643017,0.691223,0.728113,0.752938,0.729519,0.659402,0.574370,0.474245,0.402940,0.461294,...,0.706268,0.650218,0.727416,0.746537,0.766283,0.514789,0.453669,0.783242,1.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X,0.837912,0.841972,0.834226,0.824933,0.831612,0.839788,0.870319,0.903772,0.898621,0.864341,...,0.865397,0.904360,0.899706,0.823247,0.759462,0.871792,0.843129,0.648951,0.868968,1
X,0.899518,0.880000,0.910408,0.940430,0.971388,0.982522,1.000000,0.991855,0.951647,0.871412,...,0.716792,0.626567,0.738240,0.610874,0.727605,0.757862,0.657690,0.792556,1.000000,1
X,0.710817,0.720143,0.704892,0.617803,0.528379,0.414255,0.327710,0.363878,0.465357,0.553734,...,0.665386,0.824555,0.954049,0.786360,0.568688,0.665561,0.715077,0.735498,0.935721,1
X,0.853779,0.741087,0.661940,0.728122,0.807062,0.846544,0.849598,0.851051,0.886479,0.899977,...,0.564531,0.756740,0.831966,0.748112,0.770145,0.705628,0.857763,0.867560,0.873760,1


In [60]:
label_mapping_festival

{'MI': 2, 'Normal': 3, 'Abnormal': 0, 'History_MI': 1}

# Doing testing dataset

In [62]:
def process_images_in_directory(directory, target):
    df_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            image_path = os.path.join(directory, filename)
            image = imread(image_path)
            df = image_to_dataframe(image)
            
            # Add a "target" column with the specified value
            df['target'] = target
            
            df_list.append(df)
    return pd.concat(df_list, axis=0)

# Directories
MI_dir = "/Users/omgoswami/Desktop/ECG_stuffs/test/ECG Images of Myocardial Infarction Patients (240x12=2880)"
Norm_dir = "/Users/omgoswami/Desktop/ECG_stuffs/test/Normal Person ECG Images (284x12=3408)"
Abnorm_dir = "/Users/omgoswami/Desktop/ECG_stuffs/test/ECG Images of Patient that have abnormal heartbeat (233x12=2796)"
History_MI_dir = "/Users/omgoswami/Desktop/ECG_stuffs/test/ECG Images of Patient that have History of MI (172x12=2064)"

final_df_test = pd.DataFrame()
directories = [(MI_dir, "MI"), (Norm_dir, "Normal"), (Abnorm_dir, "Abnormal"), (History_MI_dir, "History_MI")]

for directory, target in directories:
    df_result = process_images_in_directory(directory, target)
    final_df_test = pd.concat([final_df_test, df_result], axis=0)

# Display the final_df
final_df_test

,0,1,2,3,4,5,6,7,8,9,...,3306,3307,3308,3309,3310,3311,3312,3313,3314,target
X,0.596155,0.610622,0.603065,0.595087,0.570312,0.604930,0.683520,0.779340,0.871618,0.949770,...,0.580224,0.611294,0.748822,0.934437,0.931842,0.714574,0.586167,0.598591,0.770899,MI
X,0.643017,0.691223,0.728113,0.752938,0.729519,0.659402,0.574370,0.474245,0.402940,0.461294,...,0.706268,0.650218,0.727416,0.746537,0.766283,0.514789,0.453669,0.783242,1.000000,MI
X,0.747484,0.754454,0.751588,0.727533,0.695317,0.642815,0.599851,0.584066,0.612402,0.648837,...,0.786704,0.402675,0.321619,0.341773,0.367131,0.299174,0.437112,0.615842,0.906445,MI
X,0.658102,0.690143,0.667195,0.575440,0.471922,0.360966,0.352923,0.466738,0.574876,0.695192,...,0.524878,0.607964,0.572406,0.612392,0.511942,0.200321,0.301086,0.556108,0.871898,MI
X,0.557265,0.494912,0.438873,0.384765,0.369616,0.422018,0.474093,0.471702,0.495931,0.542497,...,0.407439,0.609470,0.801359,0.587813,0.507644,0.548047,0.441984,0.568155,0.896731,MI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X,0.975180,0.971437,0.899557,0.819248,0.733103,0.652111,0.564536,0.482561,0.481708,0.563702,...,0.849892,0.814170,0.786113,0.858991,0.767880,0.572602,0.381285,0.186191,0.003009,History_MI
X,0.780885,0.814398,0.831585,0.860685,0.851792,0.857066,0.848633,0.857220,0.833556,0.777355,...,0.371457,0.775778,0.789528,0.708458,0.762648,0.800525,0.840988,0.813280,0.920730,History_MI
X,0.551613,0.490181,0.398624,0.301365,0.221572,0.267887,0.362967,0.433446,0.518508,0.603462,...,0.888806,0.952470,0.611149,0.472576,0.529032,0.577633,0.581136,0.687554,0.965995,History_MI
X,0.682510,0.682286,0.641051,0.620212,0.608210,0.576331,0.603596,0.645714,0.677964,0.720297,...,0.882263,0.900776,0.860660,0.727438,0.819492,0.903790,0.732192,0.520632,0.841832,History_MI


In [63]:
df_copy_test = final_df_test.copy()

In [64]:
final_df_test.to_csv('final_df_test.csv',index=False)

In [65]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_column = label_encoder.fit_transform(df_copy_test['target'])
label_mapping_test = {original_value: encoded_value for original_value, encoded_value in zip(df_copy_test['target'], encoded_column)}
df_copy_test['target'] = encoded_column
df_copy_test

,0,1,2,3,4,5,6,7,8,9,...,3306,3307,3308,3309,3310,3311,3312,3313,3314,target
X,0.596155,0.610622,0.603065,0.595087,0.570312,0.604930,0.683520,0.779340,0.871618,0.949770,...,0.580224,0.611294,0.748822,0.934437,0.931842,0.714574,0.586167,0.598591,0.770899,2
X,0.643017,0.691223,0.728113,0.752938,0.729519,0.659402,0.574370,0.474245,0.402940,0.461294,...,0.706268,0.650218,0.727416,0.746537,0.766283,0.514789,0.453669,0.783242,1.000000,2
X,0.747484,0.754454,0.751588,0.727533,0.695317,0.642815,0.599851,0.584066,0.612402,0.648837,...,0.786704,0.402675,0.321619,0.341773,0.367131,0.299174,0.437112,0.615842,0.906445,2
X,0.658102,0.690143,0.667195,0.575440,0.471922,0.360966,0.352923,0.466738,0.574876,0.695192,...,0.524878,0.607964,0.572406,0.612392,0.511942,0.200321,0.301086,0.556108,0.871898,2
X,0.557265,0.494912,0.438873,0.384765,0.369616,0.422018,0.474093,0.471702,0.495931,0.542497,...,0.407439,0.609470,0.801359,0.587813,0.507644,0.548047,0.441984,0.568155,0.896731,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X,0.975180,0.971437,0.899557,0.819248,0.733103,0.652111,0.564536,0.482561,0.481708,0.563702,...,0.849892,0.814170,0.786113,0.858991,0.767880,0.572602,0.381285,0.186191,0.003009,1
X,0.780885,0.814398,0.831585,0.860685,0.851792,0.857066,0.848633,0.857220,0.833556,0.777355,...,0.371457,0.775778,0.789528,0.708458,0.762648,0.800525,0.840988,0.813280,0.920730,1
X,0.551613,0.490181,0.398624,0.301365,0.221572,0.267887,0.362967,0.433446,0.518508,0.603462,...,0.888806,0.952470,0.611149,0.472576,0.529032,0.577633,0.581136,0.687554,0.965995,1
X,0.682510,0.682286,0.641051,0.620212,0.608210,0.576331,0.603596,0.645714,0.677964,0.720297,...,0.882263,0.900776,0.860660,0.727438,0.819492,0.903790,0.732192,0.520632,0.841832,1


In [66]:
label_mapping_test

{'MI': 2, 'Normal': 3, 'Abnormal': 0, 'History_MI': 1}

# PCA

In [69]:
X_train = df_copy.iloc[:,:-1]
Y_train = df_copy.iloc[:,-1]
X_test = df_copy_test.iloc[:,:-1]
Y_test = df_copy_test.iloc[:,-1]

X_test

,0,1,2,3,4,5,6,7,8,9,...,3305,3306,3307,3308,3309,3310,3311,3312,3313,3314
X,0.596155,0.610622,0.603065,0.595087,0.570312,0.604930,0.683520,0.779340,0.871618,0.949770,...,0.888907,0.580224,0.611294,0.748822,0.934437,0.931842,0.714574,0.586167,0.598591,0.770899
X,0.643017,0.691223,0.728113,0.752938,0.729519,0.659402,0.574370,0.474245,0.402940,0.461294,...,0.608713,0.706268,0.650218,0.727416,0.746537,0.766283,0.514789,0.453669,0.783242,1.000000
X,0.747484,0.754454,0.751588,0.727533,0.695317,0.642815,0.599851,0.584066,0.612402,0.648837,...,0.910572,0.786704,0.402675,0.321619,0.341773,0.367131,0.299174,0.437112,0.615842,0.906445
X,0.658102,0.690143,0.667195,0.575440,0.471922,0.360966,0.352923,0.466738,0.574876,0.695192,...,0.615814,0.524878,0.607964,0.572406,0.612392,0.511942,0.200321,0.301086,0.556108,0.871898
X,0.557265,0.494912,0.438873,0.384765,0.369616,0.422018,0.474093,0.471702,0.495931,0.542497,...,0.461301,0.407439,0.609470,0.801359,0.587813,0.507644,0.548047,0.441984,0.568155,0.896731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X,0.975180,0.971437,0.899557,0.819248,0.733103,0.652111,0.564536,0.482561,0.481708,0.563702,...,0.817112,0.849892,0.814170,0.786113,0.858991,0.767880,0.572602,0.381285,0.186191,0.003009
X,0.780885,0.814398,0.831585,0.860685,0.851792,0.857066,0.848633,0.857220,0.833556,0.777355,...,0.512083,0.371457,0.775778,0.789528,0.708458,0.762648,0.800525,0.840988,0.813280,0.920730
X,0.551613,0.490181,0.398624,0.301365,0.221572,0.267887,0.362967,0.433446,0.518508,0.603462,...,0.605255,0.888806,0.952470,0.611149,0.472576,0.529032,0.577633,0.581136,0.687554,0.965995
X,0.682510,0.682286,0.641051,0.620212,0.608210,0.576331,0.603596,0.645714,0.677964,0.720297,...,0.891928,0.882263,0.900776,0.860660,0.727438,0.819492,0.903790,0.732192,0.520632,0.841832


In [70]:
from sklearn.decomposition import PCA

#do PCA and choose componeents as 400
pca = PCA(n_components=400)
x_pca = pca.fit_transform(X_train)
x_pca = pd.DataFrame(x_pca)

# Calculate the variance explained by priciple components
explained_variance = pca.explained_variance_ratio_
print('Variance of each component:', pca.explained_variance_ratio_)
print('\n Total Variance Explained:', round(sum(list(pca.explained_variance_ratio_))*100, 2))

Variance of each component: [8.47910621e-02 4.45917558e-02 3.55638892e-02 2.82720839e-02
 2.45129942e-02 2.20102797e-02 2.05139530e-02 1.92712515e-02
 1.85309094e-02 1.71224115e-02 1.54964008e-02 1.44723381e-02
 1.41403370e-02 1.36981215e-02 1.29782042e-02 1.27405997e-02
 1.23024618e-02 1.21048586e-02 1.12503356e-02 1.09911599e-02
 1.03668546e-02 1.00879313e-02 9.80296678e-03 9.61252654e-03
 9.55727254e-03 9.25905124e-03 8.69332393e-03 8.35792057e-03
 8.19215779e-03 8.13898880e-03 7.79153573e-03 7.70430909e-03
 7.42940607e-03 7.11338193e-03 6.93819233e-03 6.74428697e-03
 6.57367087e-03 6.43548987e-03 6.12684047e-03 6.03912488e-03
 5.88961308e-03 5.87096871e-03 5.71460811e-03 5.63269506e-03
 5.50061254e-03 5.40445702e-03 5.28285212e-03 5.12307642e-03
 4.87884107e-03 4.79603526e-03 4.73945248e-03 4.65607864e-03
 4.47521521e-03 4.41402043e-03 4.26613296e-03 4.22002085e-03
 4.15338171e-03 4.05484603e-03 3.99580411e-03 3.93212376e-03
 3.89044768e-03 3.81763711e-03 3.71299043e-03 3.69104682e

In [71]:
x_pca

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,-3.767981,-1.822450,-7.939919,2.714556,-1.495143,-1.957810,1.669407,5.988840,-5.431438,-0.782836,...,-0.035092,0.045715,0.033223,-0.021803,-0.051083,-0.017418,-0.023609,0.004650,0.040651,-0.015573
1,3.534515,0.726371,-1.993918,3.658657,-1.477098,2.978997,-1.997583,0.317088,0.813647,-0.417085,...,0.017453,-0.035993,0.047690,-0.035007,0.019375,-0.029073,-0.024679,-0.012484,0.024075,0.007381
2,0.199453,-2.124527,0.348214,-4.959754,-0.522114,-1.313911,-0.602109,1.936786,-2.974699,1.932917,...,-0.029767,0.026577,-0.011663,0.018804,0.001166,0.025862,-0.050967,0.009121,-0.037115,0.018748
3,-7.618703,-1.064228,-1.247279,3.322158,1.839760,3.748141,0.178694,-3.274372,-0.561393,-2.037825,...,-0.003202,0.067207,-0.033185,0.070717,-0.041302,0.068222,-0.015511,0.037658,-0.013153,-0.012389
4,3.212448,-0.550566,-3.355994,3.844658,4.469722,2.756056,-0.402223,2.890793,2.506684,1.145324,...,-0.035864,-0.044160,0.013657,-0.018648,0.000688,-0.007099,0.005159,0.027388,-0.000892,0.011895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,1.857361,-0.383453,8.370426,-0.476820,1.475139,-1.205424,-0.729794,0.452675,-1.244777,-1.402116,...,-0.008435,-0.189359,0.205686,0.285914,-0.038229,0.114624,0.024270,-0.070251,0.121036,-0.033532
924,4.704571,-0.811370,5.539445,3.069208,1.211418,1.960650,-2.512219,-1.433317,-1.023033,-0.067028,...,0.175669,-0.018531,0.158387,0.049421,-0.232465,-0.039887,-0.272283,0.080182,-0.160063,-0.091794
925,0.449754,4.265885,2.462222,-0.659509,1.987180,0.574616,0.903302,-0.190262,0.541229,0.196792,...,-0.015303,-0.100057,-0.031908,0.088920,-0.064998,-0.036242,0.000861,-0.045869,-0.128807,-0.029666
926,2.476868,4.063778,1.763213,0.068122,0.228656,0.616716,-4.499272,0.878361,-0.343523,0.144633,...,0.085261,-0.077232,0.144238,0.135045,-0.007472,0.072141,0.133738,-0.106565,-0.077710,-0.056508


In [72]:
from sklearn.decomposition import PCA

#do PCA and choose componeents as 400
pca = PCA(n_components=400)
x_pca_test = pca.fit_transform(X_test)
x_pca_test = pd.DataFrame(x_pca_test)

x_pca_test

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,-1.239269,-0.395127,0.452989,-1.115808,-1.558680,1.072672,-3.249605,1.929233,-0.442587,-1.193830,...,-5.143804e-16,-1.116544e-15,3.229465e-16,-1.851200e-16,-3.404129e-16,-3.455867e-17,3.555104e-17,2.933709e-18,-6.575047e-17,2.149844e-16
1,2.805997,-1.194091,-2.679274,-3.828856,5.458358,-1.528361,-1.423285,2.927877,1.047602,-0.563111,...,4.743368e-16,3.425487e-16,-5.241688e-16,-1.477725e-17,6.038455e-17,-1.374553e-16,-6.585903e-17,-1.541057e-16,-1.475761e-16,-4.138447e-16
2,8.646747,-0.512427,1.153473,-2.813409,-2.820017,-3.746366,-0.042873,-1.259837,-1.540365,1.116362,...,-4.579795e-16,5.727487e-16,8.502359e-16,-6.135864e-16,2.721632e-16,-1.577697e-16,-6.587441e-16,1.367285e-16,-1.270252e-16,5.647415e-16
3,4.817861,-2.582621,0.720790,4.722348,-1.134688,-0.907312,0.596703,1.054511,0.224548,-4.234684,...,-5.942885e-16,6.645700e-16,7.774209e-16,5.108789e-16,1.418783e-18,-2.033737e-16,1.531686e-16,-3.003800e-17,4.698610e-16,1.560053e-16
4,5.631113,2.281931,-2.689396,0.858734,-0.651557,-1.249358,2.163415,-1.037869,0.555820,1.751162,...,-6.889313e-16,6.750052e-16,1.866604e-16,-3.997079e-16,1.515694e-16,-1.282266e-16,-1.036575e-15,7.066730e-16,-2.598368e-16,-2.234187e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,-3.697544,3.629973,-0.128386,1.413607,-0.353119,2.032444,2.367675,-1.800185,2.434835,-1.938478,...,2.989123e-18,1.505614e-18,-2.613001e-18,-9.058707e-19,5.412829e-19,-2.100433e-18,2.429642e-18,-7.457947e-19,1.463699e-18,4.055862e-19
444,-0.037391,-0.935209,1.821755,-2.032499,2.594116,1.931069,0.244869,0.650837,0.043605,0.808058,...,2.989123e-18,1.505614e-18,-2.613001e-18,-9.058707e-19,5.412829e-19,-2.100433e-18,2.429642e-18,-7.457947e-19,1.463699e-18,4.055862e-19
445,6.423610,0.444546,-0.471548,0.079914,1.937523,-1.700853,-1.322650,0.329467,0.489451,-0.565557,...,2.989123e-18,1.505614e-18,-2.613001e-18,-9.058707e-19,5.412829e-19,-2.100433e-18,2.429642e-18,-7.457947e-19,1.463699e-18,4.055862e-19
446,-2.555368,7.299761,0.893812,-0.037243,-0.485868,0.812055,-0.492968,-1.195523,1.570004,1.661616,...,2.989123e-18,1.505614e-18,-2.613001e-18,-9.058707e-19,5.412829e-19,-2.100433e-18,2.429642e-18,-7.457947e-19,1.463699e-18,4.055862e-19
